```
# 1. 修改列名
# df.columns = ['A','B','C']
# df.rename(columns={'a':'A', 'b':'B', 'c':'C'}, inplace = True)
#修改指定列名
# df.rename(columns={'a':'A', 'c':'C'}, inplace = True)

# 2. 修改数据类型
df['Customer Number'].astype("int")
#  这样的操作并没有改变原始的数据框，而只是返回的一个拷贝

# 想要真正的改变数据框，通常需要通过赋值来进行，比如
df["Customer Number"] = df["Customer Number"].astype("int")

# 当然可以通过lambda 函数将这个比较简单的函数一行带过
df["2016"].apply(lambda x: x.replace(",","").replace("$","")).astype("float64")
df['date'] = df['date'].astype('datetime64[ns]')
pd.to_datetime("")
pd.to_datetime("2020-07-06")
```

In [1]:
from bs4 import BeautifulSoup  as bs
import requests
import os
import pandas as pd



In [5]:
def Crawling_data(lottery="dlt"):
    """
    # url = "http://datachart.500.com/dlt/history/newinc/history.php?start=00001&end=99999"
    # 大乐透 http://datachart.500.com/dlt/history/history.shtml
    # 大乐透：第一列期号 第二到第六列 5个前区号码  第七到第八列 2个后区号码  最后一列（15）开奖日期
    <tr class="t_tr1"><td class="t_tr1">20058</td><td class="cfont2">01</td><td class="cfont2">15</td><td class="cfont2">19</td><td class="cfont2">26</td><td class="cfont2">27</td><td class="cfont4">05</td><td class="cfont4">10</td><td class="t_tr1">801,073,309</td><td class="t_tr1">5</td><td class="t_tr1">9,421,778</td><td class="t_tr1">137</td><td class="t_tr1">80,612</td><td class="t_tr1">311,403,757</td><td class="t_tr1">2020-07-04</td></tr>


    # url = "http://datachart.500.com/ssq/history/newinc/history.php?start=00001&end=99999"
    # 双色球 http://datachart.500.com/ssq/history/history.shtml
    # 双色球：第一列期号 第二到第七列 6个红球号码  第八列蓝球号码  最后一列（16）开奖日期
    <tr class="t_tr1"><td>20059</td><td class="t_cfont2">02</td><td class="t_cfont2">04</td><td class="t_cfont2">10</td><td class="t_cfont2">17</td><td class="t_cfont2">22</td><td class="t_cfont2">25</td><td class="t_cfont4">14</td><td class="t_cfont4">&nbsp;</td><td>1,002,178,704</td><td>4</td><td>10,000,000</td><td>190</td><td>147,308</td><td>386,807,160</td><td>2020-07-05</td></tr>
    """
    remove = -1
    url = "http://datachart.500.com/dlt/history/newinc/history.php?start=00001&end=99999"
    if lottery == "ssq":
        url = "http://datachart.500.com/ssq/history/newinc/history.php?start=00001&end=99999"
        remove = 8  # 移除 快乐星期天 的那一列

    data = requests.get(url).text
    data = bs(data, 'lxml')

    data = data.find("tbody", id="tdata").find_all('tr', **{"class": "t_tr1"})

    lo_data = []
    for content in data:
        row = []
        tds = content.find_all('td')
        for i in range(0, len(tds)):
            if i != remove:
                row.append(tds[i].get_text())
        lo_data.append(row)

    return pd.DataFrame(lo_data, columns= ["no",
                                           "num1","num2","num3","num4","num5","num6","num7",
                                           "bonus_pool",
                                           "one_count","one_bonus",
                                           "two_count","two_bonus",
                                           "sale",
                                           "date"])


In [20]:
dlt_df = Crawling_data("dlt")
ssq_df = Crawling_data("ssq")


In [21]:
dlt_df.columns,dlt_df.dtypes



(Index(['no', 'num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'num7',
        'bonus_pool', 'one_count', 'one_bonus', 'two_count', 'two_bonus',
        'sale', 'date'],
       dtype='object'),
 no            object
 num1          object
 num2          object
 num3          object
 num4          object
 num5          object
 num6          object
 num7          object
 bonus_pool    object
 one_count     object
 one_bonus     object
 two_count     object
 two_bonus     object
 sale          object
 date          object
 dtype: object)

In [22]:
def dfastype(df):
    for c in ["no","num1","num2","num3","num4","num5","num6","num7"]:
        df[c] = df[c].astype("str")
    for c in ["one_count","two_count"]:
        df[c] = df[c].astype("int")
    for c in ["bonus_pool","one_bonus","two_bonus","sale"]:
        df[c] = df[c].apply(lambda x: x.replace(",","")).astype("float64")
    for c in ["date"]:
        df[c] = df[c].astype("datetime64[ns]")

In [23]:
dfastype(dlt_df)
dfastype(ssq_df)

dlt_df.columns,dlt_df.dtypes



(Index(['no', 'num1', 'num2', 'num3', 'num4', 'num5', 'num6', 'num7',
        'bonus_pool', 'one_count', 'one_bonus', 'two_count', 'two_bonus',
        'sale', 'date'],
       dtype='object'),
 no                    object
 num1                  object
 num2                  object
 num3                  object
 num4                  object
 num5                  object
 num6                  object
 num7                  object
 bonus_pool           float64
 one_count              int32
 one_bonus            float64
 two_count              int32
 two_bonus            float64
 sale                 float64
 date          datetime64[ns]
 dtype: object)

In [24]:
# dlt_df.to_csv("大乐透.csv")
# ssq_df.to_csv("双色球.csv")
